This notebook fine-tunes GTP-NeoX-20B on the Alpaca dataset.
You can find a full description of the process here: [Fine-tune a Chat Model on Your Data with QLoRa](https://open.substack.com/pub/kaitchup/p/fine-tune-a-chat-model-on-your-data-with-qlora-7bf89810f9b?r=2kp66c&utm_campaign=post&utm_medium=web)

If you have any questions, please leave them as comments on the article. I'll answer them quickly.

This notebook should work on a free instance of Google Colab but it reaches the limit of the VRAM. I recommend a V100 (16 GB of VRAM) instance to be sure that you won't be out-of-memory.

First install these dependices:

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.2 MB/s eta 0:00:00


Load the tokenizer and set up a PAD token:

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "EleutherAI/gpt-neox-20b"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

#gpt-neox-20b doesn't have a PAD token
#We can use the EOS token for padding
tokenizer.pad_token = tokenizer.eos_token

We need a custom data collator to prepare Alpaca.

In [ ]:
from datasets import load_dataset, Dataset
import torch, copy
import transformers
from typing import Optional, Dict, Sequence
from torch.nn.utils.rnn import pad_sequence
from dataclasses import dataclass, field

IGNORE_INDEX = -100


@dataclass
class DataCollatorForCausalLM(object):
    tokenizer: transformers.PreTrainedTokenizer
    source_max_len: int
    target_max_len: int

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        sources = [f"{self.tokenizer.bos_token}{example['input']}" for example in instances]
        targets = [f"{example['output']}{self.tokenizer.eos_token}" for example in instances]
        # Tokenize
        tokenized_sources_with_prompt = self.tokenizer(
            sources,
            max_length=self.source_max_len,
            truncation=True,
            add_special_tokens=False,
        )
        tokenized_targets = self.tokenizer(
            targets,
            max_length=self.target_max_len,
            truncation=True,
            add_special_tokens=False,
        )
        # Build the input and labels for causal LM
        input_ids = []
        labels = []
        for tokenized_source, tokenized_target in zip(
            tokenized_sources_with_prompt['input_ids'],
            tokenized_targets['input_ids']
        ):
          input_ids.append(torch.tensor(tokenized_source + tokenized_target))
          labels.append(
              torch.tensor([IGNORE_INDEX for _ in range(len(tokenized_source))] + copy.deepcopy(tokenized_target))
          )


        # Apply padding
        input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)
        labels = pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        data_dict = {
            'input_ids': input_ids,
            'attention_mask':input_ids.ne(0),
        }

        if labels is not None:
            data_dict['labels'] = labels
        return data_dict

data_collator = DataCollatorForCausalLM(
    tokenizer=tokenizer,
    source_max_len=1024,
    target_max_len=256,
)

This is the format for Alpaca:

In [ ]:
ALPACA_PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response: "
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response: "
    ),
}

def extract_alpaca_dataset(example):
    if example.get("input", "") != "":
        prompt_format = ALPACA_PROMPT_DICT["prompt_input"]
    else:
        prompt_format = ALPACA_PROMPT_DICT["prompt_no_input"]
    return {'input': prompt_format.format(**example)}

We get the dataset from the HF Hub and remove all the column that we won't use:

In [ ]:
alpaca_data = load_dataset("tatsu-lab/alpaca")
alpaca_data = alpaca_data.map(extract_alpaca_dataset, remove_columns=['instruction'])

#We keep only the modified input and output columns in the dataset
alpaca_data = alpaca_data.remove_columns(
      [col for col in alpaca_data.column_names['train'] if col not in ['input', 'output']]
)
my_data = alpaca_data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/tatsu-lab___parquet/tatsu-lab--alpaca-2b32f0433506ef5f/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

Prepare the QLoRa configuration and download the model. This is a big model so it can take some time.

In [ ]:
#QLoRa configuration
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, device_map={"":0}, use_cache=False)

Loading checkpoint shards:   0%|          | 0/46 [00:00<?, ?it/s]

I enable gradient checkpointing to reduce VRAM usage. With an A100 or an RTX 24 GB, you can skip this step for a much faster fine-tuning.

In [ ]:
model.gradient_checkpointing_enable()

We add the LoRa parameters to GPT NeoX:

In [ ]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

Then, we fine-tune the model. The hyperparameter are set for demonstration purpose. Once you have confirmed it works, I recommend:



*  Higher batch size if you have enough VRAM
*  Replace "max_steps=10" by "num_train_epochs=1" to be sure that the entire dataset is seen at least once during training
*  warmup_steps=100 (you can experiment different values to see what works best)



In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=my_data['train'],
    data_collator=data_collator,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        remove_unused_columns=False

    )
)
trainer.train()

Step,Training Loss
1,4.183600
2,3.018700
3,2.186700
4,3.930200
5,2.143400
6,1.728900
7,2.055100
8,2.141500
9,2.251500
10,1.199700


TrainOutput(global_step=10, training_loss=2.4839205384254455, metrics={'train_runtime': 71.7044, 'train_samples_per_second': 0.558, 'train_steps_per_second': 0.139, 'total_flos': 280545522278400.0, 'train_loss': 2.4839205384254455, 'epoch': 0.0})